##### The Jester Dataset
###### Jester is a dataset developed by Ken Goldberg and his group at UC Berkeley. The dataset contains jokes rated by users on a scale of -10 to 10.

##### Getting the Data
###### The version of Jester dataset is provided for this assignment: jester-data-3.csv. It contains 1048575 entries, columns; user_id, joke_id, rating

##### Custom Code
###### The custom packages; soft_impute and functionsCF will need to be installed

In [2]:
import numpy as np
import pandas as pd
from fancyimpute import BiScaler
from soft_impute import SoftImpute
from functionsCF import GenerateTrainingSet

##### Create the incomplete matrices for training and testing

In [3]:
# Read jester data from files- point to where data is stored, small set of jester-data-3.csv dataset
# 100836 (rows), userId	jokeId	rating	timestamp (columns)
# read in values only
# name the variable "rating"
# YOUR CODE HERE
rating = pd.read_csv('jester-data-3.csv', sep=',').values
#raise NotImplementedError()

In [4]:
# Show the top 5 rows
print(rating[:5, :])

[[ 1.    1.   -7.82]
 [ 1.    2.    8.79]
 [ 1.    3.   -9.66]
 [ 1.    4.   -8.16]
 [ 1.    5.   -7.52]]


######  Use all known information to create the incomplete matrix

In [6]:
# First, create an empty matrix call it "matrix_incomplete"
# YOUR CODE HERE
matrix_incomplete = np.zeros((len(np.unique(rating[:,0])), len(np.unique(rating[:,1]))))

#raise NotImplementedError()

In [ ]:
# Second, Since some jokes may not have any ratings, we only use the jokes that have ratings. 
# Here we correspondingly change the joke IDs to make each column have ratings.
# create an array of all joke IDs call it "usedID"
# YOUR CODE HERE
#raise NotImplementedError()
usedID = np.unique(rating[:, 1])

# replace the joke IDs by their positions in the array we just created
# YOUR CODE HERE
#raise NotImplementedError()
for i in range(len(rating[:,1])):
    rating[:,1][i] = np.where(usedID==rating[:,1][i])[0][0] + 1


In [8]:
# Finally, we construct the incomplete matrix, on which the incomplete components are nan by 
# default. 
# Make all components of matrix_incomplete nan by default
# YOUR CODE HERE
#raise NotImplementedError()
matrix_incomplete[:] = np.nan


# create the index pair of the components with ratings. Call it "indices"
# YOUR CODE HERE
#raise NotImplementedError()
indices = np.array(rating[:,0] - 1).astype(int), np.array(rating[:,1] - 1).astype(int)


# change the values in the corresponding positions to the known rating information
# YOUR CODE HERE
#raise NotImplementedError()
matrix_incomplete[indices] = rating[:,2]


In [9]:
# Obtain the index pairs of the training set and the validation set, with ratio 80%
# Use GenerateTrainingSet() function and name the outputs train_indices and validation_indices
# YOUR CODE HERE
#raise NotImplementedError()
train_indices, validation_indices = GenerateTrainingSet(rating[:,0], rating[:,1], 0.80)


# Now use the index pairs to create the incomplete training test called "matrix_train"
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train = matrix_incomplete.copy()
matrix_train[:] = np.nan
matrix_train[train_indices] = matrix_incomplete[train_indices]


##### Run the softImpute model for collaborative filtering

In [10]:
# Create the BiScaler model call it "biscaler"
# YOUR CODE HERE
#raise NotImplementedError()
biscaler = BiScaler(scale_rows=False, scale_columns=False, max_iters=50, verbose=False)


# Rescale both rows and columns to have zero mean
# Call it matrix_train_normalized
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_normalized = biscaler.fit_transform(matrix_train)


In [11]:
# Use softImpute to complete the matrix. J means the number of archetypes and rand_seed means the 
# seed for the inner random number generator, verbose control whether outputting algorithm logs.

softImpute = SoftImpute(J = 9, maxit = 200, random_seed = 2022, verbose = False)


In [12]:
# Run the softImpute model on the normalized training set
# Call the output matrix_train_softImpute
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_softImpute = softImpute.fit(matrix_train_normalized)


# Use the softImpute model to create the predicted matrix. If we set copyto as True, then it 
# directly change the value of matrix_train_normalized
# Call the output matrix_train_filled_normalized
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_filled_normalized = matrix_train_softImpute.predict(matrix_train_normalized, copyto = False)


# Inverse transformation to undo the scaling
# Call the output matrix_train_filled
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_filled = biscaler.inverse_transform(matrix_train_filled_normalized)


##### Analysis of the predicted ratings

###### Out-of-sample R^2

In [13]:
# Create the baseline method of the training set using np.average()
# Name the output train_average
# YOUR CODE HERE
#raise NotImplementedError()
train_average = np.average(matrix_train[train_indices])


In [ ]:
# Calculate out-of-sample R2 and in-sample R2
# Your results may vary from the lesson due to datasize and training test split.
# Generate validation_mse, training_mse, validation_mse_baseline, and training_mse_baseline
# YOUR CODE HERE
#raise NotImplementedError()
validation_mse = ((matrix_train_filled[validation_indices] - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse = ((matrix_train_filled[train_indices] - matrix_incomplete[train_indices]) ** 2).mean()
validation_mse_baseline = ((train_average - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse_baseline = ((train_average - matrix_incomplete[train_indices]) ** 2).mean()

print("out-of-sample R2: %.4f, in-sample R2: %.4f." % (1 - validation_mse / validation_mse_baseline, 1 - training_mse / training_mse_baseline))

In [ ]:
# Observe your prediction:
# After matrix completion
print("After matrix completion =", matrix_train_filled[10882,64])